In [ ]:
# import cv2 as cv
# import numpy as np
# import matplotlib.pyplot as plt
# import mss.tools
# import time
# import uuid
# import keyboard

# def drive ():
#     time.sleep(10)
#     with mss.mss() as sct:
#         monitor_info = sct.monitors[0]
#         top_margin = (monitor_info["height"] // 3) + 50
#         monitor = {"top": monitor_info["top"] + top_margin, "left": monitor_info["left"] + 55, "width": monitor_info["width"] - 120, "height": (monitor_info["height"] * 2 // 3)-105}

#         # hold space for a bit when you're stopping so it stops
#         while not keyboard.is_pressed("space"):
#             output = f"testingDataset/{str(uuid.uuid4())}.png".format(**monitor)
#             sct_img = sct.grab(monitor)
#             mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
#             print(f"Captured {output}")
#             time.sleep(1)

    

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import mss.tools
import time
import uuid
import keyboard
import math
import pyautogui
import time

def drive():
    img_orig = cv.imread('dataset/fd7f4e1f-1905-47ca-b00c-b2d1c6c7fd98.png')
    img_rgb = cv.cvtColor(img_orig, cv.COLOR_BGR2RGB)
    img_blur = cv.blur(img_orig, ksize=(5, 5))
    img_canny = cv.Canny(img_blur, threshold1=121.6, threshold2=180.9)
    cdst = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
    linesP = cv.HoughLinesP(img_canny, 1, np.pi / 250, 90, None, 94.6, 20.5)
    
    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(cdst, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv.LINE_AA)
    
    image_width = 1800
    threshold = image_width // 2  # Half of the image width

    # Separate left and right coordinates based on the threshold
    left_y = []
    left_x = []
    right_y = []
    right_x = []

    for line in linesP:
        x1, y1, x2, y2 = line[0]
        # Separate coordinates into left and right sides
        if x1 < threshold and x2 < threshold:
            left_y.extend([y1, y2])
            left_x.extend([x1, x2])
        elif x1 >= threshold and x2 >= threshold:
            right_y.extend([y1, y2])
            right_x.extend([x1, x2])

    # Find the lowest Y coordinates on the left and right sides
    if left_y:
        left_lowest_index = left_y.index(min(left_y))
        x1_left_lowest, y1_left_lowest = left_x[left_lowest_index], left_y[left_lowest_index]
        cv.circle(cdst, (x1_left_lowest, y1_left_lowest), 10, (0, 0, 255), -1)  # Mark left lowest point in red

    if right_y:
        right_lowest_index = right_y.index(min(right_y))
        x2_right_lowest, y2_right_lowest = right_x[right_lowest_index], right_y[right_lowest_index]
        cv.circle(cdst, (x2_right_lowest, y2_right_lowest), 10, (0, 0, 255), -1)  # Mark right lowest point in red

    # Calculate Euclidean distance
    distance = math.sqrt((x2_right_lowest - x1_left_lowest)**2 + (y2_right_lowest - y1_left_lowest)**2)

    print("Distance between the points:", distance)

    center = distance / 2
    print('center:', center)

    desired_x_point = x1_left_lowest + center
    mario_x_point = 900

    print('Desired x point:', desired_x_point)
    print(desired_x_point - mario_x_point)


    if(desired_x_point - mario_x_point) > 0:
        label = 'right'
    else:
        label = 'left'

    print(label)
    #cv.circle(cdst, (992, 300), 10, (0, 0, 255), -1)  # Mark left lowest point in red
    return label

    #cv.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdst)
    #cv.waitKey()

label = drive()
def move(label):
    time.sleep(10)
    if (label == 'right'):
        pyautogui.keyDown('D')
        pyautogui.mouseDown()
        pyautogui.sleep(10)
        pyautogui.keyUp('D')
        pyautogui.mouseUp()

move(label)